In [2]:
import torch.nn as nn

In [3]:
import pandas as pd
import torch
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [114]:
df = pd.read_csv('/Users/mazelenova/Downloads/merged_genotypes.csv', index_col=0) 
df.head()

,Пол,Порода,Дата_Рождения,Родитель_папа,Родитель_мама,Удой л/день,Упитанность,Коэффициент инбридинга (F),Прирост веса кг/день,Здоровье (1-10),...,Генетическая ценность (баллы),chrom,pos,ref,alt,Признак,beta,Генотип коровы,Genotype_Effect,SNP_Contribution
ID_особи,,,,,,,,,,,,,,,,,,,,,
1,Самка,Симентальская,2022-12-22,1996,150,NaN,4,0.17,NaN,8,...,93,4,39128359,G,T,Фертильность (%),9.52,T/T,0.0,0.000
1,Самка,Симентальская,2022-12-22,1996,150,NaN,4,0.17,NaN,8,...,93,6,5624123,T,A,Прирост веса кг/день,-6.76,A/A,0.0,-0.000
1,Самка,Симентальская,2022-12-22,1996,150,NaN,4,0.17,NaN,8,...,93,26,41747388,T,A,Здоровье (1-10),-2.64,T/T,0.0,-0.000
2,Самка,Герефорд,2018-02-07,7154,6500,37.478242,4,0.15,1.18,5,...,87,27,25531753,C,G,Коэффициент инбридинга (F),1.04,C/C,0.0,0.000
2,Самка,Герефорд,2018-02-07,7154,6500,37.478242,4,0.15,1.18,5,...,87,105,9093860,T,C,Упитанность,0.15,T/C,0.5,0.075


In [115]:
df = df.reset_index()
df.head()

,ID_особи,Пол,Порода,Дата_Рождения,Родитель_папа,Родитель_мама,Удой л/день,Упитанность,Коэффициент инбридинга (F),Прирост веса кг/день,...,Генетическая ценность (баллы),chrom,pos,ref,alt,Признак,beta,Генотип коровы,Genotype_Effect,SNP_Contribution
0,1,Самка,Симентальская,2022-12-22,1996,150,NaN,4,0.17,NaN,...,93,4,39128359,G,T,Фертильность (%),9.52,T/T,0.0,0.000
1,1,Самка,Симентальская,2022-12-22,1996,150,NaN,4,0.17,NaN,...,93,6,5624123,T,A,Прирост веса кг/день,-6.76,A/A,0.0,-0.000
2,1,Самка,Симентальская,2022-12-22,1996,150,NaN,4,0.17,NaN,...,93,26,41747388,T,A,Здоровье (1-10),-2.64,T/T,0.0,-0.000
3,2,Самка,Герефорд,2018-02-07,7154,6500,37.478242,4,0.15,1.18,...,87,27,25531753,C,G,Коэффициент инбридинга (F),1.04,C/C,0.0,0.000
4,2,Самка,Герефорд,2018-02-07,7154,6500,37.478242,4,0.15,1.18,...,87,105,9093860,T,C,Упитанность,0.15,T/C,0.5,0.075


In [118]:
# Save the trained model
model_path = "/Users/mazelenova/Downloads/breeding_model.pth"
model.load_state_dict(torch.load(model_path))
model.eval()  # Set the model to evaluation mode
print("Model loaded successfully.")


Model loaded successfully.


In [119]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Predict on the test set
with torch.no_grad():
    y_pred = model(X_test).numpy()
    y_true = y_test.numpy()

# Calculate metrics
mse = mean_squared_error(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

print(f"Evaluation Metrics:")
print(f"  Mean Squared Error (MSE): {mse:.4f}")
print(f"  Mean Absolute Error (MAE): {mae:.4f}")
print(f"  R² Score: {r2:.4f}")


Evaluation Metrics:
  Mean Squared Error (MSE): 0.0002
  Mean Absolute Error (MAE): 0.0114
  R² Score: 1.0000


In [21]:
# Example: Predict for the first 5 samples in the test set
example_inputs = X_test[:5]

with torch.no_grad():
    predictions = model(example_inputs)
print(f"Predictions: {predictions.numpy().flatten()}")


Predictions: [88.004486 81.51472  90.00626  71.0156   74.51481 ]


In [18]:
def predict_for_animal(animal_id, df, scaler, model):
    """
    Predict compatibility scores for a specific cow ID with all bulls.

    Args:
        animal_id (int): The ID of the cow to predict for.
        df (pd.DataFrame): The dataset containing all animals.
        scaler (scaler): The StandardScaler used during training.
        model (torch.nn.Module): The trained model.

    Returns:
        pd.DataFrame: Compatibility scores for all bulls.
    """
    # Find the selected cow's data
    cow_data = df[df['ID_особи'] == animal_id]
    if cow_data.empty:
        raise ValueError(f"Cow ID {animal_id} not found in the dataset.")

    # Filter for bulls (opposite gender)
    cow_gender = cow_data['Пол'].iloc[0]
    candidates = df[df['Пол'] != cow_gender]

    # Calculate pairwise features
    pairwise_features = []
    for _, candidate in candidates.iterrows():
        avg_genetic_value = (cow_data['Генетическая ценность (баллы)'].values[0] +
                             candidate['Генетическая ценность (баллы)']) / 2
        pairwise_features.append([
            avg_genetic_value,
            cow_data['Коэффициент инбридинга (F)'].values[0],
            candidate['Коэффициент инбридинга (F)']
        ])

    # Scale features
    pairwise_features = scaler.transform(pairwise_features)

    # Convert to tensor
    pairwise_tensor = torch.tensor(pairwise_features, dtype=torch.float32)

    # Predict compatibility scores
    with torch.no_grad():
        compatibility_scores = model(pairwise_tensor).numpy().flatten()

    # Add scores to the candidates DataFrame
    candidates = candidates.copy()
    candidates['compatibility_score'] = compatibility_scores

    return candidates[['ID_особи', 'compatibility_score']].sort_values(by='compatibility_score', ascending=False)


def show_top_matches_unique(animal_id, df, scaler, model, top_n=5):
    """
    Show the top N unique compatibility matches for a given cow ID.

    Args:
        animal_id (int): The ID of the cow to predict for.
        df (pd.DataFrame): The dataset containing all animals.
        scaler (scaler): The StandardScaler used during training.
        model (torch.nn.Module): The trained model.
        top_n (int): Number of top matches to show.

    Returns:
        pd.DataFrame: Top N unique matches as cow-bull pairs with compatibility scores.
    """
    try:
        # Predict all compatibility scores for the animal
        top_matches = predict_for_animal(animal_id, df, scaler, model)

        # Drop duplicate entries based on 'ID_особи', keeping the highest score
        top_matches_unique = top_matches.drop_duplicates(subset='ID_особи')

        # Sort by compatibility score in descending order and get the top N
        top_matches_unique = top_matches_unique.sort_values(by='compatibility_score', ascending=False).head(top_n)

        print(f"Top {top_n} Unique Matches for Cow ID {animal_id}:")
        return top_matches_unique
    except ValueError as e:
        print(e)
        return pd.DataFrame()





In [16]:
df.reset_index(inplace=True)
df.head()

,ID_особи,Упитанность,Коэффициент инбридинга (F),Прирост веса кг/день,Генетическая ценность (баллы),Genotype_Effect,SNP_Contribution,Пол
0,2,4,0.15,1.18,87,0.0,0.000,0
1,2,4,0.15,1.18,87,0.5,0.075,0
2,2,4,0.15,1.18,87,0.0,-0.000,0
3,2,4,0.15,1.18,87,0.5,-4.325,0
4,8,3,0.15,1.16,90,0.0,0.000,1


In [14]:
# Predict for a chosen cow ID
animal_id = 1994  # Replace with your cow ID
top_n = 5  # Number of top matches to display

In [24]:
# Example: Predict for Cow ID 1994
animal_id = 3556
top_n = 5

# Show top unique matches
unique_matches = show_top_matches_unique(animal_id, df, scaler, model, top_n)
if not unique_matches.empty:
    print(unique_matches)



Top 5 Unique Matches for Cow ID 3556:
      ID_особи  compatibility_score
9990      9023            87.511086
3890      3569            87.511086
1284      1173            87.511086
7098      6455            87.510918
1341      1225            87.510918


In [121]:
def find_high_compatibility_pairs(df, scaler, model, threshold=0.8):
    """
    Find all pairs of animals in the dataset with high compatibility scores.

    Args:
        df (pd.DataFrame): The dataset containing all animals.
        scaler (scaler): The StandardScaler used during training.
        model (torch.nn.Module): The trained model.
        threshold (float): The minimum compatibility score to be considered "high".

    Returns:
        pd.DataFrame: A DataFrame of high-compatibility pairs sorted by score.
    """
    # Separate animals by gender
    males = df[df['Пол'] == 1]
    females = df[df['Пол'] == 0]

    # Generate all possible pairs
    pairs = pd.merge(males, females, how='cross', suffixes=('_male', '_female'))

    # Calculate pairwise features
    pairwise_features = []
    for _, pair in pairs.iterrows():
        avg_genetic_value = (pair['Генетическая ценность (баллы)_male'] + pair['Генетическая ценность (баллы)_female']) / 2
        pairwise_features.append([
            avg_genetic_value,
            pair['Коэффициент инбридинга (F)_male'],
            pair['Коэффициент инбридинга (F)_female']
        ])

    # Scale features
    pairwise_features = np.array(pairwise_features)
    pairwise_features_scaled = scaler.transform(pairwise_features)

    # Convert to PyTorch tensor
    features_tensor = torch.tensor(pairwise_features_scaled, dtype=torch.float32)

    # Predict compatibility scores
    model.eval()
    with torch.no_grad():
        compatibility_scores = model(features_tensor).numpy().flatten()

    # Add compatibility scores to the pairs DataFrame
    pairs = pairs.copy()
    pairs['compatibility_score'] = compatibility_scores

    # Filter high-compatibility pairs
    high_compatibility_pairs = pairs[pairs['compatibility_score'] >= threshold]

    # Sort by compatibility score
    high_compatibility_pairs = high_compatibility_pairs.sort_values(by='compatibility_score', ascending=False)

    return high_compatibility_pairs
